In [1]:
import pandas as pd

In [2]:
final_data = pd.read_csv('part_3_RFM_SCORE_sales_data_with_customer_RFM_score_nov_27_final.csv')

In [3]:
final_data.head()

,invoiceno,code,quantity,year,invoicedate,unitprice,sellprice,custid,country,shipper,...,month,count,class,class_type,invent_class,R,F,M,score,category
0,568200,PADS,5,2017,2017-09-25,0.999975,4.999876,16198,United Kingdom,Delivery Truck,...,9,1,C,CZ,red,4,4,4,444,Hibernating
1,568375,BANK CHARGES,5,2017,2017-09-26,0.999976,4.999883,13405,United Kingdom,UPS Next day Air,...,9,1,C,CZ,red,2,2,2,222,Highvalue
2,561226,PADS,5,2017,2017-07-26,0.999886,4.999428,15618,United Kingdom,Delivery Truck,...,7,1,C,CZ,red,4,2,4,424,Highvalue
3,2550193,PADS,7,2018,2018-06-08,0.999962,6.999731,13952,United Kingdom,UPS ground 2nd day,...,6,1,C,CZ,red,2,2,1,221,Highvalue
4,550193,PADS,5,2017,2017-04-15,0.999988,4.999940,13952,United Kingdom,UPS ground 2nd day,...,4,1,C,CZ,red,2,2,1,221,Highvalue



## Feature development 

#### For a customer id: Final  invent_class is the maxmimum frequency a particular invent class was order

#### Feature 1: Invent_class per customer_id

In [4]:
t1 = final_data.groupby(['custid','invent_class'],as_index=False).agg({'quantity': lambda x: x.sum()})

In [5]:
t1.head()

,custid,invent_class,quantity
0,12346,green,664240
1,12347,green,10942
2,12347,red,3114
3,12347,yellow,6116
4,12348,green,16826


In [6]:
t2 = t1.groupby('custid',as_index=False).agg({'quantity': lambda x: x.max()})
t2.rename(columns={'quantity':'max_quantity'},inplace=True)

In [7]:
t2.head()

,custid,max_quantity
0,12346,664240
1,12347,10942
2,12348,16826
3,12349,490
4,12350,597


In [8]:
class_dist = pd.merge(t2,t1,how='left',left_on=['custid','max_quantity'],right_on=['custid','quantity'])

In [9]:
final_invent_class_label = class_dist[['custid','invent_class']]

In [10]:
len(final_invent_class_label)

4152

In [11]:
## Sorting  dataframe

final_invent_class_label.sort_values(by='custid',inplace=True)

D:\ArcGIS\a\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
final_invent_class_label.invent_class.value_counts()

green     3202
yellow     665
red        285
Name: invent_class, dtype: int64

#### Feature 2: Product portfolio: Out of total 5 commodities, which customer orders maximum type of  commodities

In [13]:
product_porfolio = final_data.groupby(['custid'],as_index=False).agg({'commodity': lambda x:  len(x.unique())})
product_porfolio.rename(columns={'commodity':'product_portfolio'},inplace=True)

In [14]:
len(product_porfolio)

4146

In [15]:
## Sorting dataframe
product_porfolio.sort_values(by='custid',inplace=True)

#### Feature 3: History_length , Customer association with the company

In [16]:
final_data['invoicedate'] = pd.to_datetime(final_data['invoicedate'])

In [17]:
final_data['mix_date']=final_data['invoicedate']
final_data['max_date']=final_data['invoicedate']

In [18]:
dates_data=final_data.groupby('custid',as_index=False).agg({'mix_date': lambda x: x.min(),'max_date': lambda x: x.max()})

In [19]:
dates_data['length_history'] = (dates_data['max_date']-dates_data['mix_date']).dt.days

In [20]:
len(dates_data)

4146

In [21]:
dates_data[dates_data['length_history']>30].count()

custid            3054
mix_date          3054
max_date          3054
length_history    3054
dtype: int64

In [22]:
quantiles_history = dates_data['length_history'].quantile(q=[0.25,0.50,0.75])

In [23]:
def hist_score(x):
    if x <= quantiles_history[0.25]:
        return 4
    elif x <= quantiles_history[0.50]:
        return 3
    elif x <= quantiles_history[0.75]:
        return 2
    else:
        return 1


In [24]:
dates_data['length_history'] = dates_data['length_history'].apply( lambda x: hist_score(x))

In [25]:
len(dates_data)

4146

In [26]:
## Sorting dataframe
dates_data.sort_values(by='custid',inplace=True)

##### Feature 4: R F M

In [27]:
rfm_date = final_data.groupby(['custid'],as_index=False).agg({'R': 'mean','F': 'mean','M': 'mean'})

In [28]:
len(rfm_date)

4146

In [29]:
## Sorting dataframe
rfm_date.sort_values(by='custid',inplace=True)

## Merging all feature together 

In [30]:
final1_2 = pd.merge(dates_data,rfm_date,on='custid',how='left')

In [31]:
final1_2.head()

,custid,mix_date,max_date,length_history,R,F,M
0,12346,2017-01-18,2018-05-09,3,3,4,1
1,12347,2016-12-07,2018-12-06,1,1,1,1
2,12348,2016-12-16,2018-12-15,1,1,1,1
3,12349,2017-11-21,2017-11-21,4,3,4,3
4,12350,2017-02-02,2018-08-01,2,3,4,3


In [32]:
final1_2 = pd.merge(final1_2,product_porfolio,on='custid',how='left')

In [33]:
final1_2.head()

,custid,mix_date,max_date,length_history,R,F,M,product_portfolio
0,12346,2017-01-18,2018-05-09,3,3,4,1,1
1,12347,2016-12-07,2018-12-06,1,1,1,1,5
2,12348,2016-12-16,2018-12-15,1,1,1,1,5
3,12349,2017-11-21,2017-11-21,4,3,4,3,5
4,12350,2017-02-02,2018-08-01,2,3,4,3,5


In [34]:
final1_2 = pd.merge(final1_2,final_invent_class_label,on='custid',how='left')

In [35]:
final = final1_2[['custid','length_history','R','F','M','product_portfolio','invent_class']]

In [36]:
final.head()

,custid,length_history,R,F,M,product_portfolio,invent_class
0,12346,3,3,4,1,1,green
1,12347,1,1,1,1,5,green
2,12348,1,1,1,1,5,green
3,12349,4,3,4,3,5,green
4,12350,2,3,4,3,5,green


In [37]:
final.to_csv('part_4_Customer_pre_final_NOV27.csv',index=False)

###  weightage for attributes in individual features
#### IF R = 1 , weightage for attribut R1 = 1
#### similary if R=2 , weightage for attribute R2 = 0.8
#### and so on 

#### 'R': {1: 1.0, 2: 0.8, 3: 0.5, 4: 0.1}
    
#### 'F': {1: 1.0, 2: 0.8, 3: 0.5,4: 0.1}
 
#### 'M': {1: 1.0, 2: 0.8, 3: 0.5,4: 0.1}

#### 'product_portfolio': {5: 1.0, 4: 0.9, 3: 0.8, 2: 0.7, 1 : 0.6}
    
#### 'invent_class': {'green':1.0, 'yellow': 0.8 ,'red': 0.5}
    
#### 'length_history':{1: 1.0, 2: 0.8, 3: 0.6, 4: 0.4}


In [38]:
final_labels = {'R': {1: 1.0, 2: 0.8, 3: 0.5, 4: 0.1},
 'F': {1: 1.0, 2: 0.8, 3: 0.5,4: 0.1},
 'M': {1: 1.0, 2: 0.8, 3: 0.5,4: 0.1},
 'product_portfolio': {5: 1.0, 4: 0.9, 3: 0.8, 2: 0.7, 1 : 0.6},
 'invent_class': {'green':1.0, 'yellow': 0.8 ,'red': 0.5},
 'length_history':{1: 1.0, 2: 0.8, 3: 0.6, 4: 0.4}}




In [39]:
def final_scaling(x,key1):
    weights={'R':0.15,'M':0.25,'F':0.25,'product_portfolio':0.15,'invent_class':0.10,'length_history':0.10}
    current_weight = weights[key1]
    return current_weight*(final_labels[key1][x])

In [40]:
final_weighted = final.copy()

In [41]:
final_weighted.product_portfolio.isnull().sum()

0

In [42]:
final_weighted['F_weighted'] = final_weighted['F'].apply(final_scaling,args=('F'))
final_weighted['M_weighted'] = (final_weighted['M'].apply(final_scaling,args=('M')))
final_weighted['R_weighted'] = (final_weighted['R'].apply(final_scaling,args=('R')))
# product_portfolio = (trial3['product_portfolio'].apply(final_scaling,args=['product_portfolio']))
# invent_class = (trial3['invent_class'].apply(final_scaling,args=('invent_class')))
# length_history = (trial3['length_history'].apply(final_scaling,args=('length_history')))

# final['R_factored'] = final['R'].apply(lambda x: final_scaling('R',x))
# final['M_factored'] = final['M'].apply(lambda x: final_scaling('M',x))
# final['factored_product_portfolio'] = final['product_portfolio'].apply(lambda x: final_scaling('product_portfolio',x))
# final['factored_invent_class'] = final['invent_class'].apply(lambda x: final_scaling('invent_class',x))
# final['length_history'] = final['length_history'].apply(lambda x: final_scaling('length_history',x))

In [43]:
final_weighted['product_portfolio_weighted']= final_weighted['product_portfolio'].apply(lambda x: final_scaling(x,'product_portfolio'))
final_weighted['invent_class_weighted']= final_weighted['invent_class'].apply(lambda x: final_scaling(x,'invent_class'))
final_weighted['length_history_weighted'] = final_weighted['length_history'].apply(lambda x: final_scaling(x,'length_history'))


In [44]:
final_weighted.head()

,custid,length_history,R,F,M,product_portfolio,invent_class,F_weighted,M_weighted,R_weighted,product_portfolio_weighted,invent_class_weighted,length_history_weighted
0,12346,3,3,4,1,1,green,0.025,0.250,0.075,0.09,0.1,0.06
1,12347,1,1,1,1,5,green,0.250,0.250,0.150,0.15,0.1,0.10
2,12348,1,1,1,1,5,green,0.250,0.250,0.150,0.15,0.1,0.10
3,12349,4,3,4,3,5,green,0.025,0.125,0.075,0.15,0.1,0.04
4,12350,2,3,4,3,5,green,0.025,0.125,0.075,0.15,0.1,0.08


In [45]:
final_weighted['final_value'] = final_weighted['product_portfolio_weighted']+final_weighted['invent_class_weighted']+final_weighted['length_history_weighted']+final_weighted['F_weighted']+final_weighted['M_weighted']+final_weighted['R_weighted']

In [46]:
final_weighted.head()

,custid,length_history,R,F,M,product_portfolio,invent_class,F_weighted,M_weighted,R_weighted,product_portfolio_weighted,invent_class_weighted,length_history_weighted,final_value
0,12346,3,3,4,1,1,green,0.025,0.250,0.075,0.09,0.1,0.06,0.600
1,12347,1,1,1,1,5,green,0.250,0.250,0.150,0.15,0.1,0.10,1.000
2,12348,1,1,1,1,5,green,0.250,0.250,0.150,0.15,0.1,0.10,1.000
3,12349,4,3,4,3,5,green,0.025,0.125,0.075,0.15,0.1,0.04,0.515
4,12350,2,3,4,3,5,green,0.025,0.125,0.075,0.15,0.1,0.08,0.555


#### Mapping Final value 
#### IF final value os <=0.5 = 0 or Bad performance
#### and IF value >0.5  map it to 1 or Good performance

In [47]:
def performance_map(x):
    if x<=0.6:
        response = 0 #'Bad'
        return response
    else:
        response = 1# 'Good'
        return response

In [48]:
final_weighted['customer_type'] = final_weighted['final_value'].map(performance_map)

#### Distribution of GOOD and Bad

In [49]:
final_weighted.customer_type.value_counts()

1    2419
0    1733
Name: customer_type, dtype: int64

In [50]:
len('product_portfolio')

17

In [51]:
final_weighted.head()

,custid,length_history,R,F,M,product_portfolio,invent_class,F_weighted,M_weighted,R_weighted,product_portfolio_weighted,invent_class_weighted,length_history_weighted,final_value,customer_type
0,12346,3,3,4,1,1,green,0.025,0.250,0.075,0.09,0.1,0.06,0.600,0
1,12347,1,1,1,1,5,green,0.250,0.250,0.150,0.15,0.1,0.10,1.000,1
2,12348,1,1,1,1,5,green,0.250,0.250,0.150,0.15,0.1,0.10,1.000,1
3,12349,4,3,4,3,5,green,0.025,0.125,0.075,0.15,0.1,0.04,0.515,0
4,12350,2,3,4,3,5,green,0.025,0.125,0.075,0.15,0.1,0.08,0.555,0


In [52]:
## Exporting File with 

final_weighted.to_csv('part_4_Customer_GOOD_BAD_parameter_nov_final.csv',index=False)